In [21]:
from openai import OpenAI
from dotenv import dotenv_values
import json
# **** GPT_MODEL **** #
GPT_MODEL = "gpt-3.5-turbo-1106"


OPENAI_API_KEY:str = dotenv_values(".env").get("OPENAI_API_KEY")
client:OpenAI = OpenAI(api_key=OPENAI_API_KEY)


In [22]:
def show_json(obj):
    return json.loads(obj.model_dump_json())

In [23]:
file = client.files.create(
    file=open("zia_profile.pdf", "rb"),
    purpose="assistants"
)

display(show_json(file))

{'id': 'file-VoylOxXdcrWCUVOv6ZugxvTo',
 'bytes': 48802,
 'created_at': 1702503247,
 'filename': 'zia_profile.pdf',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

In [24]:
from openai.types.beta.assistant import Assistant
# You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia 
assistant:Assistant = client.beta.assistants.create(
    name="Student Support Assistant",
    instructions="You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia",
    model=GPT_MODEL,
    tools=[{"type":"retrieval"}],
    file_ids=[file.id]
)


display(show_json(assistant))

{'id': 'asst_tjpRjo2JFM2gAEyaGjz53pjP',
 'created_at': 1702503249,
 'description': None,
 'file_ids': ['file-VoylOxXdcrWCUVOv6ZugxvTo'],
 'instructions': 'You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Student Support Assistant',
 'object': 'assistant',
 'tools': [{'type': 'retrieval'}]}

In [25]:
from openai.types.beta.thread import Thread
thread:Thread =  client.beta.threads.create()

In [26]:
from openai.types.beta.threads.thread_message import ThreadMessage

message:ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="When and which city Zia U. Khan was born?",
)

display(show_json(assistant))


{'id': 'asst_tjpRjo2JFM2gAEyaGjz53pjP',
 'created_at': 1702503249,
 'description': None,
 'file_ids': ['file-VoylOxXdcrWCUVOv6ZugxvTo'],
 'instructions': 'You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Student Support Assistant',
 'object': 'assistant',
 'tools': [{'type': 'retrieval'}]}

In [27]:
from openai.types.beta.threads.run import Run
run:Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Pakistani. The user is the student of PIAIC."
)
display(show_json(run))


{'id': 'run_niI6qb8ByGXePdOBraEfNtlU',
 'assistant_id': 'asst_tjpRjo2JFM2gAEyaGjz53pjP',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702503252,
 'expires_at': 1702503852,
 'failed_at': None,
 'file_ids': ['file-VoylOxXdcrWCUVOv6ZugxvTo'],
 'instructions': 'Please address the user as Pakistani. The user is the student of PIAIC.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_bd0LKzQbVZcEilHFIgKxtjE3',
 'tools': [{'type': 'retrieval'}]}

In [28]:
run:Run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)
display(show_json(run))



{'id': 'run_niI6qb8ByGXePdOBraEfNtlU',
 'assistant_id': 'asst_tjpRjo2JFM2gAEyaGjz53pjP',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702503252,
 'expires_at': 1702503852,
 'failed_at': None,
 'file_ids': ['file-VoylOxXdcrWCUVOv6ZugxvTo'],
 'instructions': 'Please address the user as Pakistani. The user is the student of PIAIC.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1702503252,
 'status': 'in_progress',
 'thread_id': 'thread_bd0LKzQbVZcEilHFIgKxtjE3',
 'tools': [{'type': 'retrieval'}]}

In [30]:
messages:list[ThreadMessage] =  client.beta.threads.messages.list(
    thread_id=thread.id
)

for m in reversed(messages.data):
    print(m.role + " : "+m.content[0].text.value)



user : When and which city Zia U. Khan was born?
assistant : I'll check the file to find the information about Zia U. Khan's birth.
